In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from datetime import timedelta
import joblib
from datetime import date
import ruptures as rpt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import warnings
from scipy.stats import pearsonr
import os
from sklearn.model_selection import RepeatedKFold
from datetime import datetime, timedelta

In [ ]:
co_data=pd.read_csv('CO_2018_2025.csv',index_col=0)

In [ ]:
co_data=co_data.dropna()

,city,date,t2m,wd,ws,rh,msl,mtpr,blh,tcc,...,total_stations,percentage_data,CO,day_julian,hour,weekday,date_unix,year,week,month
0,Delhi,2018-05-01 00:00:00,303.04544,117.600290,4.840165,52.410230,100260.090,0.000000,353.387880,0.250568,...,31,93.548387,678.103448,121,0,2,1525132800,2018,18,5
1,Delhi,2018-05-01 01:00:00,303.00570,114.175705,5.031793,52.974354,100269.340,0.000000,358.905120,0.006760,...,31,93.548387,674.051724,121,1,2,1525136400,2018,18,5
2,Delhi,2018-05-01 02:00:00,301.43796,113.780530,5.025594,57.611480,100237.650,0.000000,367.615900,0.000000,...,31,87.096774,382.685185,121,2,2,1525140000,2018,18,5
3,Delhi,2018-05-01 03:00:00,301.22278,114.315560,4.859068,58.139008,100258.430,0.000000,387.548000,0.000847,...,31,90.322581,553.214286,121,3,2,1525143600,2018,18,5
4,Delhi,2018-05-01 04:00:00,299.54980,109.940125,4.769044,67.338030,100229.664,0.000000,408.764530,0.413689,...,31,90.322581,575.565476,121,4,2,1525147200,2018,18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53322,Delhi,2025-02-28 19:00:00,296.92212,146.961140,2.137012,71.840230,100983.160,6.123161,80.548645,0.196472,...,31,87.096774,1251.505141,59,19,5,1740769200,2025,9,2
53323,Delhi,2025-02-28 20:00:00,294.63245,129.123210,2.125412,83.814804,101042.550,26.186085,68.574000,0.193956,...,31,87.096774,1284.409621,59,20,5,1740772800,2025,9,2
53324,Delhi,2025-02-28 21:00:00,293.02722,118.093590,2.470502,94.192890,101149.810,37.578392,85.552390,0.239538,...,31,90.322581,1327.028504,59,21,5,1740776400,2025,9,2
53325,Delhi,2025-02-28 22:00:00,292.87674,102.703950,2.995194,94.603000,101226.080,19.602442,118.071700,0.249462,...,31,90.322581,1292.976195,59,22,5,1740780000,2025,9,2


In [ ]:
seed = np.random.RandomState(0)

In [ ]:
columns = ['t2m','msl','blh','ssr','mtpr','tcc','wd','ws','rh','hour','day_julian','weekday','date_unix']

In [ ]:
delhi_predictors, delhi_CO = co_data[columns], co_data['CO']

In [ ]:
delhi_predictors_train, delhi_predictors_test, delhi_CO_train, delhi_CO_test = train_test_split(delhi_predictors, delhi_CO, test_size=0.3,random_state=seed)

In [ ]:
gbdt= HistGradientBoostingRegressor(random_state=seed, n_iter_no_change=5, early_stopping=True)
pipeline = Pipeline(steps=[ ("gbdt", gbdt)])
params = {
    "gbdt__max_iter": np.arange(100,500,25),
    "gbdt__learning_rate": np.arange(0.01,0.1,0.03),
    "gbdt__max_depth": np.arange(2,7,1),
    "gbdt__min_samples_leaf": np.arange(1, 20, 1),
}
cv_gbdt = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
search = RandomizedSearchCV(pipeline, param_distributions=params, n_jobs=-1, cv=cv_gbdt, random_state=seed, n_iter = 500)
search.fit(delhi_predictors_train, delhi_CO_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Best parameter (CV score=0.873):
{'gbdt__min_samples_leaf': 15, 'gbdt__max_iter': 425, 'gbdt__max_depth': 6, 'gbdt__learning_rate': 0.09999999999999999}


In [ ]:
params = {
    "gbdt__learning_rate": [0.099],
    "gbdt__max_depth": np.arange(4, 8, 1),
    "gbdt__min_samples_leaf": np.arange(12, 18, 1),
    "gbdt__max_iter": np.arange(375, 475, 25),
}

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid=params, n_jobs=-1, cv=cv_gbdt)
grid_search.fit(delhi_predictors_train, delhi_CO_train)
print(grid_search.best_params_)

{'gbdt__learning_rate': 0.099, 'gbdt__max_depth': 7, 'gbdt__max_iter': 450, 'gbdt__min_samples_leaf': 15}


In [ ]:
best_model = grid_search.best_estimator_

In [ ]:
joblib.dump(best_model, 'best_model_delhi_2025CO.joblib')

['best_model_delhi_2025CO.joblib']

In [ ]:
print(f'R2 score (train): {best_model.score(delhi_predictors_train, delhi_CO_train):.4f}')
print(f'R2 score (test): {best_model.score(delhi_predictors_test, delhi_CO_test):.4f}')

R2 score (train): 0.9320
R2 score (test): 0.8834


In [ ]:
CO_prediction_test = best_model.predict(delhi_predictors_test)

In [ ]:
predictions_df = pd.DataFrame({'Predicted_CO': CO_prediction_test})


In [ ]:
predictions_df['Actual_CO'] = co_data.loc[delhi_predictors_test.index, 'CO'].values
predictions_df['date'] = co_data.loc[delhi_predictors_test.index, 'date'].values

In [ ]:
predictions_df.to_csv('delhi_CO_test_predictions_2025.csv', index=False)

In [ ]:
delhi_predictors_train.to_csv("delhi_CO_predictors_train_2025.csv", index=False)
delhi_CO_train.to_csv("delhi_CO_train_2025.csv", index=False)
delhi_predictors_test.to_csv("delhi_CO_predictors_test_2025.csv", index=False)
delhi_CO_test.to_csv("delhi_CO_test_2025.csv", index=False)

In [ ]:
mae = mean_absolute_error(best_model.predict(delhi_predictors_test), delhi_CO_test)
rmse = mean_squared_error(best_model.predict(delhi_predictors_test), delhi_CO_test, squared=False)
mbe = np.mean(best_model.predict(delhi_predictors_test) - delhi_CO_test)
r = pearsonr(best_model.predict(delhi_predictors_test), delhi_CO_test)
mape = np.mean(np.abs((delhi_CO_test - best_model.predict(delhi_predictors_test)) / delhi_CO_test)) * 100
print(mae, rmse, mbe, r, mape)

167.16841315131205 251.56342419264118 0.9213703543208134 PearsonRResult(statistic=0.9400831159482266, pvalue=0.0) 12.307726850594634


/home/ceew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
nrows_data = len(co_data.index)

53327

In [ ]:
normalised_data = pd.DataFrame()
for i in range(2000):
    met_shuffled = delhi_predictors.drop(axis=1, columns=['date_unix','day_julian','weekday','hour']).sample(n=nrows_data).reset_index(drop=True);
    met_shuffled[['date_unix','day_julian','weekday','hour']] = delhi_predictors[['date_unix','day_julian','weekday','hour']];
    prediction_data = best_model.predict(met_shuffled[columns]);
    normalised_data[i] = prediction_data;
    if (i%100 == 0):
        print(i)
    warnings.filterwarnings('ignore')

In [ ]:
mean_df = normalised_data.mean(axis=1).reset_index()
mean_df.columns = ['Index', 'Mean_Value']

In [ ]:
co_data.reset_index(drop=True, inplace=True)

In [ ]:
co_data['deweathered_CO'] = mean_df['Mean_Value']
co_data['date'] = pd.to_datetime(co_data['date'])
co_data['met_CO'] = co_data['CO'] - co_data['deweathered_CO']

In [ ]:
co_data.to_csv('DeweatheredCO_2018_2025.csv')